# Model Evaluation and Hyperparameter Tuning

### Objective
- Train multiple classification models on the Titanic dataset.
- Evaluate their performance using accuracy, precision, recall, and F1-score.
- Implement hyperparameter tuning techniques like GridSearchCV and RandomizedSearchCV to optimize model parameters.
- Select the best-performing model.

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [14]:
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
# Drop unnecessary columns
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Fill missing values (fixed version)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Encode categorical features
label_enc = LabelEncoder()
df['Sex'] = label_enc.fit_transform(df['Sex'])
df['Embarked'] = label_enc.fit_transform(df['Embarked'])

# Separate features and target
X = df.drop('Survived', axis=1)
y = df['Survived']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [17]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\nModel: {name}")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))



Model: Logistic Regression
[[90 15]
 [20 54]]
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       105
           1       0.78      0.73      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.80       179
weighted avg       0.80      0.80      0.80       179


Model: Decision Tree
[[84 21]
 [19 55]]
              precision    recall  f1-score   support

           0       0.82      0.80      0.81       105
           1       0.72      0.74      0.73        74

    accuracy                           0.78       179
   macro avg       0.77      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179


Model: Random Forest
[[89 16]
 [20 54]]
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       105
           1       0.77      0.73      0.75        74

    accuracy                           0.80       179

In [18]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

print("Best parameters from GridSearchCV:")
print(grid.best_params_)
print("Best score from GridSearchCV:", grid.best_score_)

Best parameters from GridSearchCV:
{'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50}
Best score from GridSearchCV: 0.8243770314192849


In [19]:
from scipy.stats import uniform
param_dist = {
    "C": uniform(0.1, 1.0),                
    "gamma": ['scale', 'auto'],
    "kernel": ['linear', 'rbf']
}

rand = RandomizedSearchCV(
    estimator=SVC(),
    param_distributions=param_dist,
    n_iter=5,                   
    scoring='accuracy',
    cv=3,                        
    random_state=42,
    verbose=1                   
)

rand.fit(X_train, y_train)

print("\nBest parameters from RandomizedSearchCV:")
print(rand.best_params_)
print("Best score from RandomizedSearchCV:", rand.best_score_)


Fitting 3 folds for each of 5 candidates, totalling 15 fits

Best parameters from RandomizedSearchCV:
{'C': np.float64(0.47454011884736247), 'gamma': 'scale', 'kernel': 'linear'}
Best score from RandomizedSearchCV: 0.7893427413159357


## Random Forest performs better than SVC for this dataset, select Random Forest as your final model.